# 발표에 쓰이는 테이블만 만들기

## 만들어야할 것들 table 5(다중회귀인 듯),6(PEIS별 주가성장),7(포트폴리오 비교)
* 각 지표별 유의성도 체크 
* 5 --> 나중에 확인 다중회귀면 가능 유효성 확인이면 개극혐
* 6 --> 그냥 수익률 상승인지아닌지에 따라서 column 하나 만들어서 .sum이랑 groupby로 하면 댈 듯
* 7 
포트폴리오 A
--> 일단 컨센별로 구분이 필요함  groupby --> 그리고 언컨센에 대해서도 할 것임 --> 여기서 한번AR,CR,N
--> PEIS별로 다시한번 groupby가 필요한 것 같은데 --> AR, CR, N
--> 마지막에 fundamental portfolio --> 그냥 뺴기 하면댐 유효성 검증은 음.. 차이 검정인거 같은데 애매하네 무슨 검정이지?
--> 근데 내생각에는 차이검정이 거의 100% 맞다 그러니깐 년도별로 평균 수익률을 구해야하는 것 같다 
포트폴리오 B
--> 그냥 똑같음 strong buy& buy는 사고 sell은 팔고 해서 수익률이 유의한지 확인하는 것임

# Table 5 만들기
1. 일단 PanelA 는 그냥 만들고
2. 나머지는 Groupby로 만들어야 함

In [1]:
import pandas as pd
import numpy as np
from Fin import *
df = pd.read_excel("Data/Alldata.xlsx")

In [2]:
df_main_vars = df[['RNOA', 'deltaSGA', 'deltasale', 'deltaATO', 'GNOA', 'deltaACC', 'valPEIS','consen_cut']]

In [3]:
df_main_vars.describe()

,RNOA,deltaSGA,deltasale,deltaATO,GNOA,deltaACC,valPEIS,consen_cut
count,5793.000000,5793.000000,5793.000000,5793.000000,5793.000000,5793.000000,5793.000000,5793.000000
mean,-1.337618,-0.022718,0.165635,-0.015944,-0.057784,2.375564,0.000000,1.167616
std,85.442109,1.728572,4.255564,0.441024,11.769040,111.135613,1.612494,1.437237
min,-4602.057993,-125.962834,-37.665939,-12.217111,-632.874816,-1504.589832,-5.000000,0.000000
25%,-2.051286,-0.016638,-0.050928,-0.093830,-0.113680,-1.300744,-1.000000,0.000000
50%,0.137125,0.001254,0.026968,-0.009804,0.014981,0.095316,0.000000,0.000000
75%,2.059163,0.019983,0.110887,0.056010,0.181013,1.806412,1.000000,3.000000
max,1567.058129,25.349284,291.704649,12.227867,128.514831,6354.547733,6.000000,4.000000


In [4]:
df_mean = df_main_vars.mean()
df_std = np.sqrt(df_main_vars.var())
df_p20 = df_main_vars.quantile(0.2)
df_median = df_main_vars.median()
df_p80 = df_main_vars.quantile(0.8)
df_min = df_main_vars.min()
df_max = df_main_vars.max()
now = pd.concat([df_mean, df_std, df_p20, df_median, df_p80, df_min, df_max],axis=1)
now.columns = ['Mean','Std','P20','Median','P80','Min','Max']
now.drop('consen_cut',inplace=True)
now

,Mean,Std,P20,Median,P80,Min,Max
RNOA,-1.337618,85.442109,-3.047802,0.137125,2.926182,-4602.057993,1567.058129
deltaSGA,-0.022718,1.728572,-0.023425,0.001254,0.026611,-125.962834,25.349284
deltasale,0.165635,4.255564,-0.075265,0.026968,0.144159,-37.665939,291.704649
deltaATO,-0.015944,0.441024,-0.123695,-0.009804,0.080763,-12.217111,12.227867
GNOA,-0.057784,11.769040,-0.159242,0.014981,0.245186,-632.874816,128.514831
deltaACC,2.375564,111.135613,-2.039265,0.095316,2.496503,-1504.589832,6354.547733
valPEIS,0.000000,1.612494,-1.000000,0.000000,1.000000,-5.000000,6.000000


# recoommand 별 통계

In [18]:
df_recommand = df_main_vars.set_index('consen_cut')

,RNOA,deltaSGA,deltasale,deltaATO,GNOA,deltaACC,valPEIS
consen_cut,,,,,,,
0,-3.854743,0.018968,-0.000509,-0.020195,-0.043541,-2.422581,3
2,0.017899,0.195367,-0.232862,-0.016386,0.526508,0.836038,0
0,-1.030939,0.092703,-0.180697,-0.179397,0.197369,-0.125957,-1
3,2.100839,-0.005697,0.054771,0.014982,0.058078,3.173349,0
0,-5.001211,-0.030186,-0.125749,0.066940,0.028944,-19.440243,2
...,...,...,...,...,...,...,...
3,0.775449,-0.055687,0.153598,0.181132,0.106874,4.688188,2
3,4.538554,-0.080015,0.399642,-0.003419,-0.633226,33.396904,0
0,0.288718,-0.010589,0.111131,0.181702,0.340062,-1.266596,1


In [15]:
def showTable5(df):
    listnum = [i for i in range(5)] *7
    listnum.sort()
    consens = df['consen_cut'].sort_values().unique()
    table5 = []
    for i in consens:
        df_mean = df[df['consen_cut'] == i ].mean()
        df_std = np.sqrt(df[df['consen_cut'] == i ].var())
        df_p20 = df[df['consen_cut'] == i ].quantile(0.2)
        df_median = df[df['consen_cut'] == i ].median()
        df_p80 = df[df['consen_cut'] == i ].quantile(0.8)
        df_min = df[df['consen_cut'] == i ].min()
        df_max = df[df['consen_cut'] == i ].max()
        now = pd.concat([df_mean, df_std, df_p20, df_median, df_p80, df_min, df_max],axis=1)
        table5.append(now)
    table5 = pd.concat(table5)
    table5.drop('consen_cut',inplace = True)
    table5.columns = ['Mean','Std','P20','Median','P80','Min','Max']
    table5.index = [ listnum, ['RNOA','deltaSGA','deltasale','deltaATO','GNOA','deltaACC','PEIS']*5]
    
    return table5

In [19]:
showTable5(df_main_vars)

Mean         Std       P20    Median       P80          Min  \
0 RNOA       -1.893150   99.523279 -2.532304  0.028622  2.290390 -4602.057993   
  deltaSGA   -0.037699    2.234217 -0.025329  0.001841  0.029930  -125.962834   
  deltasale   0.200921    5.477027 -0.094172  0.012539  0.125919   -37.665939   
  deltaATO   -0.012885    0.483567 -0.129014 -0.009483  0.090101   -12.217111   
  GNOA       -0.257959   14.800072 -0.178935  0.002448  0.232833  -632.874816   
  deltaACC    2.688630  121.800157 -1.880977  0.108089  2.451404 -1504.589832   
  PEIS        0.070707    1.601608 -1.000000  0.000000  1.000000    -5.000000   
1 RNOA       -0.705096    3.049060 -2.543134 -0.797407  1.114480    -3.706953   
  deltaSGA    0.069585    0.016533  0.059940  0.071710  0.079656     0.052093   
  deltasale  -0.112920    0.050516 -0.139775 -0.094277 -0.082335    -0.170107   
  deltaATO   -0.338497    0.142793 -0.422044 -0.414995 -0.270250    -0.426743   
  GNOA       -0.042105    0.263472 -0.196292 -0.183063  0.083890    -0.205112   
  deltaACC    4.074874    0.377254  3.844799  4.018870  4.293747     3.728753   
  PEIS       -1.333333    0.577350 -1.600000 -1.000000 -1.000000    -2.000000   
2 RNOA       -5.411668   42.832277 -3.545574 -0.282496  1.547089  -543.229031   
  deltaSGA    0.020185    0.075886 -0.012578  0.010921  0.058981    -0.300536   
  deltasale  -0.010039    0.190106 -0.118529 -0.008394  0.082249    -0.908265   
  deltaATO   -0.088867    0.285834 -0.171665 -0.029872  0.031668    -1.659353   
  GNOA        0.095601    6.985815 -0.203790 -0.008930  0.163103   -69.466696   
  deltaACC   11.193625  154.457047 -2.326777  0.197404  2.930430  -214.771689   
  PEIS       -0.180556    1.518810 -1.000000  0.000000  1.000000    -5.000000   
3 RNOA       -0.007480   59.955170 -4.077189  0.500113  4.291657  -992.891662   
  deltaSGA   -0.002600    0.074364 -0.021489 -0.000057  0.020134    -2.127753   
  deltasale   0.125671    0.674057 -0.034056  0.051180  0.171383    -0.980140   
  deltaATO   -0.013180    0.375777 -0.107601 -0.008734  0.072016    -6.549053   
  GNOA        0.255632    3.937204 -0.128413  0.040187  0.274042   -28.797088   
  deltaACC    0.956884   84.388330 -2.240338  0.064658  2.559339 -1442.687291   
  PEIS       -0.098719    1.630879 -1.000000  0.000000  1.000000    -5.000000   
4 RNOA       -1.128395    4.258233 -2.935011 -1.128395  0.678220    -4.139421   
  deltaSGA   -0.034161    0.030490 -0.047096 -0.034161 -0.021225    -0.055720   
  deltasale   0.524843    0.421393  0.346061  0.524843  0.703624     0.226873   
  deltaATO    0.131107    0.047452  0.110975  0.131107  0.151239     0.097553   
  GNOA       -0.027447    0.191517 -0.108701 -0.027447  0.053807    -0.162870   
  deltaACC   -0.332827    3.350296 -1.754237 -0.332827  1.088584    -2.701844   
  PEIS        3.000000    2.828427  1.800000  3.000000  4.200000     1.000000   

                     Max  
0 RNOA       1092.781171  
  deltaSGA     25.349284  
  deltasale   291.704649  
  deltaATO     12.227867  
  GNOA        108.227686  
  deltaACC   6354.547733  
  PEIS          6.000000  
1 RNOA          2.389071  
  deltaSGA      0.084953  
  deltasale    -0.074374  
  deltaATO     -0.173753  
  GNOA          0.261859  
  deltaACC      4.476999  
  PEIS         -1.000000  
2 RNOA         45.128947  
  deltaSGA      0.327337  
  deltasale     0.947899  
  deltaATO      1.141728  
  GNOA         55.366292  
  deltaACC   2229.240444  
  PEIS          4.000000  
3 RNOA       1567.058129  
  deltaSGA      0.432737  
  deltasale    15.199537  
  deltaATO      6.946829  
  GNOA        128.514831  
  deltaACC   2557.411742  
  PEIS          6.000000  
4 RNOA          1.882631  
  deltaSGA     -0.012601  
  deltasale     0.822812  
  deltaATO      0.164661  
  GNOA          0.107976  
  deltaACC      2.036191  
  PEIS          5.000000

In [ ]:
df_recommand.colname = 'Points'

In [ ]:
df_recommand.groupby('consen_cut').mean()

In [ ]:
df_main_vars = df_main_vars.set_index('consen_cut')

In [ ]:
dfs = yearbaseDFs(df, years = [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020])

In [ ]:
df_main_vars.groupby(['consen_cut',['RNOA', 'deltaSGA', 'deltasale', 'deltaATO', 'GNOA', 'deltaACC', 'valPEIS','consen_cut']]).mean()